# Isotank Design for the Pathfinder Tank
## Motivation
Test article for demonstrating isotank technology

## Design Choices
### Material
Same material (6061-T6) and wall thickness (0.25") as LV4. Smaller 6" OD.


In [1]:
import math
# Tank radial dimensions
tank_id = 5.5     # in
tank_wall_t = 0.25  # in
tank_od = tank_id+2*tank_wall_t # in
tank_r = tank_od/2   # in
tank_overall_length = 12  # in
tank_c = 2*math.pi*tank_r

# Material properties for Al 6061-T6
E = 69.0 # Young's Modules, GPa

E_imp = 10000 # ksi
F_ty = 40.0 # Yield Tensile Strength - ksi
F_tu = 45.0 # Ultimate Tensile Strength - ksi

nu = 0.33 # Poisson's Ratio


### Safety Factor
Minimum safety factor for Base 11 is 1.5 (see requirement 3.2.4.15)

In [2]:
safety_factor = 1.5

### Flanges
The use of the flange can increase the strength of the isogrid, but increases machining complexity. It was decided to omit flanges for initil version of the design.


In [3]:
flanges = False

### Skin Thickness
The tank stock material is not highly precise. The part will be machined from the outside in and there is a risk of inadvertantly maching through the pipe if the skin thickness is too small. Additionally, after talking with OMIC, they thought that a skin thickness of 0.05" (50 mil) was reasonable.

In [4]:
skin_t = 0.05


### Rib Depth
The isogrid ribs are the source of the strength of the isogrid design. The depth of the rib is the primary factor is determining the strength. Thefore, for optimimum strength, the rib depth is simply the wall thickness minus the skin thickness 

In [5]:
rib_depth = tank_wall_t - skin_t - 0.05
print('Rib Depth: {:5.3f}'.format(rib_depth))

Rib Depth: 0.150


### Rib thickness
Again, after discussion with OMIC, it was decided that a rib web thickness of 0.050" (50 mil) could be reasonably achievable.

In [6]:
rib_t = 0.05


### Pattern size
Wanted the same pattern size as LV4. Since the circumfrence is 1/2 of LV4, reduced the number of radial divisios to 12

In [7]:
import math
num_radial_divisions = 12
pattern_size = tank_od*math.pi/num_radial_divisions
print('Pattern Size: {:5.3f}'.format(pattern_size))

Pattern Size: 1.571


## Tank Properties
The isogrid properties are calculated from NASA CR-124075

In [8]:
cell_height = pattern_size
alpha = (rib_t*rib_depth)/(skin_t*cell_height) # Web non-dimensional ratio, pg. 2.0.008
t_eff = skin_t*(1+alpha) # Effective thickness 
delta = rib_depth/skin_t # rib depth to skin thickness ratio, pg. 2.0.008
if flanges == False:
    beta = (3*alpha*(1+delta)**2+(1+alpha)*(1+alpha*delta**2))**0.5
    t_star = skin_t*beta/(1+alpha) # equivilent thickness, Eq. 2.5.3
    t_bar = skin_t*(1+ 3*alpha)
    E_star = E*(1+alpha)**2.0/beta # equivilent Young's Modulus Eq. 2.5.4
print('Web non-dimensional ratio, Alpha: {:5.3f}'.format(alpha))
print('Rib depth to skin thickness, Delta: {:5.3f}'.format(delta))
print('Beta: {:5.3f}'.format(beta))
print('Skin thickness, t: {:5.3f} in'.format(skin_t))
print('Effective thickness, t_eff: {:5.3f} in'.format(t_eff))
print('Equivalant monocoque thickness, t*: {:5.3f} in'.format(t_star))
print('Equivalant weight thickness, t_bar: {:5.3f} in'.format(t_bar))
print('Equivalant monocoque Young\'s Modulus, E*: {:5.3f} GPa'.format(E_star))

Web non-dimensional ratio, Alpha: 0.095
Rib depth to skin thickness, Delta: 3.000
Beta: 2.573
Skin thickness, t: 0.050 in
Effective thickness, t_eff: 0.055 in
Equivalant monocoque thickness, t*: 0.117 in
Equivalant weight thickness, t_bar: 0.064 in
Equivalant monocoque Young's Modulus, E*: 32.182 GPa


## Mass Model
Equations for calculating the mass of the isotank based on diameter and length

In [9]:
import math

# Ring mass estimate from solidworks model
ring_mass = 0.5316 #kg = 1.172 lbs
ring_circum_density = 0.5316/(math.pi*12.0) # 12.0 since 12 was the diameter for the mass estimate
ring_length = 1.0*0.0254

weld_gap_length = 1.0*0.0254

grid_length = tank_overall_length*0.0254-2*weld_gap_length-2*ring_length
grid_t_bar = t_bar*0.0254

density = 2700 #kg/m3
t = tank_wall_t*0.0254
id = tank_id*0.0254
ring_mass = 2.0*ring_circum_density*id
weld_gap_mass = 2.0*density*weld_gap_length*(math.pi*((id/2.0 + t)**2-(id/2.0)**2))
tank_wall_mass = density*grid_length*(math.pi*((id/2.0 + grid_t_bar)**2-(id/2.0)**2))

tank_mass = ring_mass + weld_gap_mass + tank_wall_mass

print('tank_mass:',tank_mass, 'kg')

tank_mass: 0.8015710548688604 kg


## Load Calculations



### Combined Compression and Bending
For the fuel tank, the cylinder is under compression and bending. Section 4.2 on page 4.2.001 (#97) covers this scenario. The example covers:
* General Instability (N_cr_1)
* Skin Buckling (N_cr_2)
* Rib Crippling (N_cr_3)

In [17]:
import math

# Inputs
F = 1.573 # Klbf ~7000 N
M = 0.265 # K in = 30 Nm

# Material
E_star_imp = E_imp*(1+alpha)**2.0/beta

# Instability
c0 = 0.397    # 4.2.005
c1 = 10.2     # 4.2.006
c2 = 0.616    # 4.2.008

# Classical Value
N_cl = (1/(3*(1-nu*nu))**0.5)*E_star_imp/1000*t_star*t_star/tank_r # k/in

N_a = F/(2*math.pi*tank_r) # k/in
N_b = M/(math.pi*tank_r**2) # k/in
N_cr = N_a + N_b

N_cr_1 = c0*E_imp*skin_t*skin_t/tank_r*beta
N_cr_2 = c1*E_imp*skin_t*(1+alpha)*(skin_t*skin_t)/(pattern_size*pattern_size)
N_cr_3 = c2*E_imp*skin_t*(1+alpha)*(rib_t*rib_t)/(rib_depth*rib_depth)

p_burst = F_tu*1000*skin_t*(1+alpha)/tank_r
p_meop = p_burst/1.5

print('General Instability - {:3.1f} k/in, {:5.1f} klbf'.format(N_cr_1, N_cr_1*tank_c))
print('Skin Buckling       - {:3.1f} k/in, {:5.1f} klbf'.format(N_cr_2, N_cr_2*tank_c))
print('Rib Cripling        - {:3.1f} k/in, {:5.1f} klbf'.format(N_cr_3, N_cr_3*tank_c))
print('P_burst             - {:5.1f} psi'.format(p_burst))
print('P_MEOP              - {:5.1f} psi'.format(p_meop))


# Determine if burst or compression critical
# Fig 4.2.3 was reconstructed and curve fits were derived
# See fig_4.2.3_reconstruction for details
fig_x = N_cr/(E_imp*tank_r)*10**7
p_0 = 1.134*(fig_x**0.595)*F_tu
#t_m = 1.650*(fig_x**0.597)*tank_r
print()
print('P_0                 - {:5.1f} psi'.format(p_0))
if(p_burst > p_0):
    print('P_burst > P_0\nDesign is Burst Critical')
else:
    print('P_0 > P_burst\nDesign is Compression Critical')

General Instability - 8.5 k/in, 160.5 klbf
Skin Buckling       - 5.7 k/in, 106.7 klbf
Rib Cripling        - 37.5 k/in, 706.7 klbf
P_burst             - 821.6 psi
P_MEOP              - 547.7 psi

P_0                 - 393.3 psi
P_burst > P_0
Design is Burst Critical


## Stress Calcs
x is the hoop coordinate, y is the longitudinal coordinate

In [11]:
Nx = p_meop*tank_r
Ny = p_meop*tank_r/2
Nxy = 0.0

sigma_x = Nx/t_eff
sigma_y = Ny/t_eff

sigma_1 = 1/(3*t_eff)*(3*Nx-Ny)
sigma_2 = 2/(3*t_eff)*(Ny+3**0.5*Nxy)
sigma_3 = 2/(3*t_eff)*(Ny-3**0.5*Nxy)

print('Nx:',Nx,'lbf/in')
print('Ny:',Ny,'lbf/in')
print('sigma_x:',sigma_x,'psi')
print('sigma_y:',sigma_y,'psi') 
print('sigma_1:',sigma_1,'psi')
print('sigma_2:',sigma_2,'psi')
print('sigma_3:',sigma_3,'psi')


Nx: 1643.239448782706 lbf/in
Ny: 821.619724391353 lbf/in
sigma_x: 29999.999999999996 psi
sigma_y: 14999.999999999998 psi
sigma_1: 24999.999999999996 psi
sigma_2: 9999.999999999998 psi
sigma_3: 9999.999999999998 psi


## Weld Strength and Safety Factor

In [12]:
import math

F_tu_welded = 27.0 # ksi
F_su_welded = 15.0 # ksi - 4043/4047 Filler Rod
#F_su_welded = 15.0 # ksi - 5554 Filler Rod

center_weld_depth = 0.25
area = math.pi*tank_od*center_weld_depth
print('weld area',area,'in2')
F_max_welded = F_tu_welded*area
print('Tensile force',F_max_welded,'klbf')
p_burst_center_weld = F_tu_welded*1000*center_weld_depth/tank_r
print('Center Weld Burst Pressure',p_burst_center_weld,'psi')
center_weld_safety_factor = p_burst_center_weld/p_meop
print('Center Weld Safety Factor',center_weld_safety_factor)


weld area 4.71238898038469 in2
Tensile force 127.23450247038662 klbf
Center Weld Burst Pressure 2250.0 psi
Center Weld Safety Factor 4.107739748458648


## Thermal Expansion (Contraction)

In [13]:
cte = 13*10**-6 # in/in F
dt = -297-70 # Room temperature to liquid oxygen temp
delta_diameter = cte*tank_od*dt
print(delta_diameter)

-0.028626
